In [61]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
# Define the dataset class
class EnergyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return X.values, y.values

In [63]:
# Load the data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [64]:
# Create the dataset
train_dataset = EnergyDataset(X_train, y_train)
val_dataset = EnergyDataset(X_val, y_val)
test_dataset = EnergyDataset(X_test, y_test)

In [65]:
# Data Loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [66]:
# Define the QCNN model with 5 hidden layers
class QCNN(nn.Module):
    def __init__(self):
        super(QCNN, self).__init__()
        
        # Input layer (24 features to 256 neurons)
        self.fc1 = nn.Linear(24, 256)
        self.bn1 = nn.BatchNorm1d(256)
        
        # Hidden layer 1 (256 to 128 neurons)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        
        # Hidden layer 2 (128 to 64 neurons)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        
        # Hidden layer 3 (64 to 32 neurons)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        
        # Hidden layer 4 (32 to 16 neurons)
        self.fc5 = nn.Linear(32, 16) 
        self.bn5 = nn.BatchNorm1d(16) 
        
        # Hidden layer 5 (16 to 8 neurons)
        self.fc6 = nn.Linear(16, 8) 
        self.bn6 = nn.BatchNorm1d(8)
        
        # Output layer (8 neurons to 1 output)
        self.fc7 = nn.Linear(8, 1)

        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        
        # Input Layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Hidden Layer 1
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        # Hidden Layer 2
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        # Hidden Layer 3
        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu(x)
        
        # Hidden Layer 4
        x = self.fc5(x) 
        x = self.bn5(x)  
        x = self.relu(x)
        
        # Hidden Layer 5
        x = self.fc6(x) 
        x = self.bn6(x)  
        x = self.relu(x)  
        
        # Output Layer
        x = self.fc7(x)
        return x

In [67]:
# Set hyperparameters
learning_rate = 0.001
n_epochs = 400

In [68]:
# Initialize model, optimizer, and loss function
model = QCNN()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [69]:
# Lists to log training and validation loss and accuracy
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [70]:
# Train the model
for epoch in range(n_epochs):
    model.train()  
    epoch_loss = 0
    for batch in train_loader:
        X, y = batch
        X = X.clone().detach().float()
        y = y.clone().detach().float()  
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Calculate average loss for the epoch
    average_loss = epoch_loss / len(train_loader)
    train_losses.append(average_loss)

    # Calculate training accuracy (R² score)
    with torch.no_grad():
        y_train_pred = model(torch.tensor(X_train.values, dtype=torch.float32)).flatten().numpy()
        r2 = r2_score(y_train.values, y_train_pred)
        accuracy = r2 * 100
        train_accuracies.append(accuracy)

    # Validate the model
    model.eval()  
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            X, y = batch
            X = X.clone().detach().float()
            y = y.clone().detach().float()  
            outputs = model(X)
            loss = loss_fn(outputs, y)
            val_loss += loss.item()
        
        average_val_loss = val_loss / len(val_loader)
        val_losses.append(average_val_loss)

        y_val_pred = model(torch.tensor(X_val.values, dtype=torch.float32)).flatten().numpy()
        val_r2 = r2_score(y_val.values, y_val_pred)
        val_accuracy = val_r2 * 100
        val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch + 1}/{n_epochs}], Train Loss: {average_loss}')


Epoch [1/400], Train Loss: 0.010032180674262394
Epoch [2/400], Train Loss: 0.0033365067067547584
Epoch [3/400], Train Loss: 0.0029201389564659005
Epoch [4/400], Train Loss: 0.0024905729663756834
Epoch [5/400], Train Loss: 0.002262585412715762
Epoch [6/400], Train Loss: 0.0021882108538660476
Epoch [7/400], Train Loss: 0.0021109728771292226
Epoch [8/400], Train Loss: 0.0019748213247810717
Epoch [9/400], Train Loss: 0.0018949165316165038
Epoch [10/400], Train Loss: 0.001941418939085211
Epoch [11/400], Train Loss: 0.0018337017699370023
Epoch [12/400], Train Loss: 0.001809046736243272
Epoch [13/400], Train Loss: 0.001710603874910749
Epoch [14/400], Train Loss: 0.001647221962596512
Epoch [15/400], Train Loss: 0.0015471580926212937
Epoch [16/400], Train Loss: 0.0015086839279488356
Epoch [17/400], Train Loss: 0.0015175656251675501
Epoch [18/400], Train Loss: 0.0015060277809777493
Epoch [19/400], Train Loss: 0.0014328977580011046
Epoch [20/400], Train Loss: 0.001409736529100113
Epoch [21/400], 

In [73]:
# Save trained model and metrics
torch.save({
    'model_state_dict': model.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'qcnn_model.pth')

In [74]:
# Test the model on the test set
model.eval()  
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

with torch.no_grad():
    y_test_pred = model(X_test_tensor).flatten().numpy()
    
    # Calculate MSE and R² score
    test_mse = mean_squared_error(y_test.values, y_test_pred)
    test_r2 = r2_score(y_test.values, y_test_pred)

    # Calculate accuracy as R² score in percentage
    test_accuracy = test_r2 * 100
    
    print(f'Mean Squared Error on test data: {test_mse}')
    print(f'R² score (Accuracy) on test data: {test_r2}')
    print(f'Accuracy: {test_accuracy}%')

Mean Squared Error on test data: 0.0005326356919208483
R² score (Accuracy) on test data: 0.9864182625254199
Accuracy: 98.64182625254199%
